In [1]:
from bubls.utils.rag_design.rag_building_blocks import RAGBuildingBlocks
import logging
import sys
from metadata import lyft_10k, wiki_public_companies, papers_ridesharing

<jemalloc>: Unsupported system page size


In [2]:
# from llama_index.core import Settings
# Settings.llm = OpenAI(temperature=0.2, model="gpt-3.5-turbo")
# Settings.embed_model = OpenAIEmbedding(name="text-embedding-ada-002")

In [3]:
logging.basicConfig(
    stream=sys.stdout,
    level=logging.ERROR,
    format="%(asctime)s - %(levelname)s - %(message)s",
)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
components_cfg = {
    "lyft_10k": lyft_10k,
    "wiki_public_companies": wiki_public_companies,
    "papers_ridesharing": papers_ridesharing,
}

In [5]:
rag = RAGBuildingBlocks(components_cfg)
rag.execute()

Loading data artifacts for lyft_10k
Loading LLM eval data for lyft_10k, train
Loading LLM eval data for lyft_10k, val
Loading LLM eval data for lyft_10k, test
Generating Nodes for lyft_10k, train
Generating Nodes for lyft_10k, val
Generating Nodes for lyft_10k, test
Loading QA pairs for lyft_10k, train
Loading QA pairs for lyft_10k, val
Loading QA pairs for lyft_10k, test
Loading engines for lyft_10k
Loading Index for lyft_10k, baseline
Generating Query Engine for lyft_10k
Generating Query Engine Tool for lyft_10k
Loading eval data for lyft_10k, train
Loading eval data for lyft_10k, val
Loading eval data for lyft_10k, test
Loading data artifacts for wiki_public_companies
Loading LLM eval data for wiki_public_companies, train
Loading LLM eval data for wiki_public_companies, val
Loading LLM eval data for wiki_public_companies, test
Generating Nodes for wiki_public_companies, train
Generating Nodes for wiki_public_companies, val
Generating Nodes for wiki_public_companies, test
Loading QA 

## Fine-tune embeddings

In [9]:
from llama_index.finetuning import SentenceTransformersFinetuneEngine

In [10]:
qa_pairs_training = rag.qa_pairs["lyft_10k"]["train"]
qa_pairs_val = rag.qa_pairs["lyft_10k"]["val"]
corpus = rag.get_corpus_from_index(rag.index["lyft_10k"]) 

In [11]:
model_id = "all-MiniLM-L6-v2"

finetune_engine = SentenceTransformersFinetuneEngine(
    qa_pairs_training,
    model_id=model_id,
    model_output_path="fine-tune-embeddings2",
    val_dataset=qa_pairs_val,
)
finetune_engine.finetune()
embed_model = finetune_engine.get_finetuned_model()

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/13 [00:00<?, ?it/s]

Iteration:   0%|          | 0/13 [00:00<?, ?it/s]

In [12]:
from bubls.utils.evaluation.from_dataframe import is_hit, reciprocal_ranking

In [14]:
index_tuned = rag.gen_index(
    "lyft_10k",
    "mini_tuned2",
    rag.nodes["lyft_10k"]["train"]+rag.nodes["lyft_10k"]["val"]+rag.nodes["lyft_10k"]["test"], 
    {"embed_model": "local:fine-tune-embeddings2"}
)
tuned_qe = index_tuned.as_query_engine()
tuned_eval_data = rag.gen_eval_data(tuned_qe, rag.qa_pairs["lyft_10k"]["val"], corpus)

tuned_eval_data["is_hit"] =  tuned_eval_data.apply(is_hit, axis=1)
tuned_eval_data["reciprocal_ranking"] =  tuned_eval_data.apply(reciprocal_ranking, axis=1)

print(tuned_eval_data[["is_hit", "reciprocal_ranking"]].mean())

Generating Index for lyft_10k


  0%|          | 0/126 [00:00<?, ?it/s]

is_hit                0.769841
reciprocal_ranking    0.714286
dtype: float64


In [21]:
index_original = rag.gen_index(
    "lyft_10k",
    "mini_original2",
    rag.nodes["lyft_10k"]["train"]+rag.nodes["lyft_10k"]["val"]+rag.nodes["lyft_10k"]["test"], 
    {"embed_model": "local:all-MiniLM-L6-v2"}
)
original_qe = index_original.as_query_engine()
original_eval_data = rag.gen_eval_data(original_qe, rag.qa_pairs["lyft_10k"]["val"], corpus)

original_eval_data["is_hit"] =  original_eval_data.apply(is_hit, axis=1)
original_eval_data["reciprocal_ranking"] =  original_eval_data.apply(reciprocal_ranking, axis=1)

print(original_eval_data[["is_hit", "reciprocal_ranking"]].mean())

is_hit                0.730159
reciprocal_ranking    0.666667
dtype: float64
